In [ ]:
!pip install --upgrade pip
!pip install ortools
from pathlib import Path
import math, pandas as pd
from collections import defaultdict
from ortools.sat.python import cp_model
import pandas as pd
import math
import csv

In [2]:
import sys
!"{sys.executable}" -m pip install ortools

In [3]:
# SCHEDULER (CP-SAT, no pandas/numpy)
# - Only x[g,d] variables (no per-team ordering, no exact travel)
# - Hard constraints: ≤1 game per team/day, ≤1 per venue/day, 3-in-4 (or soft)
# - Soft fixed-day penalties (optional)
# - Objective: minimize Cmax (+ small "earliness" and fixed-day penalties)


# paths 
BASE = Path("C://Users//Tanmay//Downloads//UNIVERSITY OF MELBOURNE STUDY FOLDER//Scheduling and Optimisation")
TEAMS_CSV = BASE / "Teams_NBA.csv"
GAMES_CSV = BASE / "Games_NBA.csv"  

# knobs
DAYS_HORIZON   = 180
HARD_3IN4      = True    # if False, we make it soft with a penalty
SOFT_3IN4_W    = 200     # only used when HARD_3IN4 is False
SOFT_FIXED     = True
SOFT_FIXED_W   = 2000
W_CMAX         = 1000    # dominate objective
W_EARLY        = 1       # tiny pull to earlier days
TIME_LIMIT_SEC = 300
NUM_WORKERS    = 8


def read_csv_lower(path: Path):
    rows=[]
    with path.open("r", newline="", encoding="utf-8") as f:
        rdr = csv.DictReader(f)
        if not rdr.fieldnames: return rows
        for raw in rdr:
            rows.append({(k or "").strip().lower(): (v.strip() if isinstance(v,str) else v)
                         for k,v in raw.items()})
    return rows

def pick_col(cols, cands, required=False):
    for c in cands:
        if c in cols: return c
    if required: raise ValueError(f"Missing required column; expected one of {cands}")
    return None

# load teams/games (no pandas) 
def load_league(teams_csv: Path):
    rows = read_csv_lower(teams_csv)
    if not rows: raise ValueError("Teams CSV is empty.")
    cols = rows[0].keys()
    team_col  = pick_col(cols, ["team","abbr","code","name"], required=True)
    venue_col = pick_col(cols, ["venue","home_venue","arena","city","home"])
    team_list = [str(r[team_col]).upper() for r in rows]
    home_venues = {str(r[team_col]).upper(): r[venue_col] for r in rows} if venue_col else {t:f"{t}_HOME" for t in team_list}
    return team_list, home_venues

def load_games_optional(games_csv: Path, team_list, home_venues):
    if not games_csv.exists():
        gid, games = 0, []
        for h in team_list:
            for a in team_list:
                if a != h:
                    games.append({"id": gid, "home": h, "away": a, "venue": home_venues[h]})
                    gid += 1
        return games, {}
    rows = read_csv_lower(games_csv)
    if not rows: raise ValueError("Games CSV is empty.")
    cols = rows[0].keys()
    home_col = pick_col(cols, ["home","home_team","host"], required=True)
    away_col = pick_col(cols, ["away","away_team","visitor"], required=True)
    venue_col = pick_col(cols, ["venue","arena","home_venue"])
    day_col   = pick_col(cols, ["day","date","day_index","slot"])
    games, fixed, gid = [], {}, 0
    for r in rows:
        h = str(r[home_col]).upper()
        a = str(r[away_col]).upper()
        if h not in team_list or a not in team_list:
            raise ValueError(f"Unknown team in games row: {h} vs {a}")
        venue = r[venue_col] if venue_col else home_venues[h]
        games.append({"id": gid, "home": h, "away": a, "venue": venue})
        if day_col and r.get(day_col, "") != "":
            try: fixed[gid] = int(float(r[day_col]))
            except: pass
        gid += 1
    return games, fixed


def sanity_check(team_list, games, FIXED, H):
    from collections import Counter
    team_day = defaultdict(lambda: defaultdict(int))
    venue_day = defaultdict(lambda: defaultdict(int))
    for g in games:
        gid = g["id"]
        if gid in FIXED:
            d = FIXED[gid]
            team_day[g["home"]][d]+=1; team_day[g["away"]][d]+=1; venue_day[g["venue"]][d]+=1
    bad_team = [(t,d,c) for t,dd in team_day.items() for d,c in dd.items() if c>1]
    bad_venue= [(v,d,c) for v,dd in venue_day.items() for d,c in dd.items() if c>1]
    if bad_team:  print("Fixed-day TEAM clashes:", bad_team)
    if bad_venue: print("Fixed-day VENUE clashes:", bad_venue)

    home_counts = Counter(g["home"] for g in games)
    away_counts = Counter(g["away"] for g in games)
    gpt = {t:home_counts.get(t,0)+away_counts.get(t,0) for t in team_list}
    lb_team = max(gpt.values()) if gpt else 0
    lb_3in4 = max(int(math.ceil(4/3*g)) for g in gpt.values()) if gpt else 0
    venues=set(g["venue"] for g in games)
    lb_venue=max(sum(1 for g in games if g["venue"]==v) for v in venues) if games else 0
    LB=max(lb_team,lb_3in4,lb_venue)
    print(f"Sanity lower bounds → team-days: {lb_team}, 3-in-4: {lb_3in4}, venue: {lb_venue}.  Horizon={H}.")
    if H<LB: print("Horizon < lower bound: increase DAYS_HORIZON.")

# FAST model & solve
def solve_fast():
    team_list, home_venues = load_league(TEAMS_CSV)
    games, FIXED = load_games_optional(GAMES_CSV, team_list, home_venues)

    max_fixed = max(FIXED.values()) if FIXED else 0
    H = max(DAYS_HORIZON, max_fixed, 1)
    D = list(range(1, H+1))
    sanity_check(team_list, games, FIXED, H)

    # index helpers
    games_of_team = {t: [] for t in team_list}
    for g in games:
        games_of_team[g["home"]].append(g["id"])
        games_of_team[g["away"]].append(g["id"])
    venues = list({g["venue"] for g in games})

    m = cp_model.CpModel()

    # x[g,d]
    x = {(g["id"], d): m.NewBoolVar(f"x_g{g['id']}_d{d}") for g in games for d in D}

    # each game scheduled once (soft fixed option)
    fix_pen = []
    for g in games:
        gid = g["id"]
        m.Add(sum(x[(gid,d)] for d in D) == 1)
        if SOFT_FIXED and gid in FIXED:
            fd = FIXED[gid]
            v = m.NewBoolVar(f"fixed_ok_g{gid}")
            m.Add(v == x[(gid, fd)])
            fix_pen.append(1 - v)
        elif (not SOFT_FIXED) and gid in FIXED:
            fd = FIXED[gid]
            for d in D: m.Add(x[(gid,d)] == (1 if d==fd else 0))

    # capacity constraints
    for t in team_list:
        for d in D:
            m.Add(sum(x[(gid,d)] for gid in games_of_team[t]) <= 1)
    for v in venues:
        for d in D:
            m.Add(sum(x[(g["id"],d)] for g in games if g["venue"]==v) <= 1)

    # 3-in-4 directly on x (no ordering vars!)
    soft_3in4_pen = []
    for t in team_list:
        T = games_of_team[t]
        for w in D[:-3]:
            window_sum = sum(x[(gid, d)] for gid in T for d in (w, w+1, w+2, w+3))
            if HARD_3IN4:
                m.Add(window_sum <= 3)
            else:
                ov = m.NewIntVar(0, 4, f"over3in4_{t}_{w}")
                m.Add(ov >= window_sum - 3)
                soft_3in4_pen.append(ov)

    # Cmax and small "earliness" term
    Cmax = m.NewIntVar(1, H, "Cmax")
    for g in games:
        gid = g["id"]
        # Cmax >= sum d*x[g,d]
        m.Add(Cmax >= sum(d * x[(gid,d)] for d in D))

    # Push earlier: sum of day indices (very small weight)
    sum_days = sum(d * x[(g["id"], d)] for g in games for d in D)

    # objective
    obj_terms = [W_CMAX * Cmax, W_EARLY * sum_days]
    if SOFT_FIXED and fix_pen:
        obj_terms.append(SOFT_FIXED_W * sum(fix_pen))
    if (not HARD_3IN4) and soft_3in4_pen:
        obj_terms.append(SOFT_3IN4_W * sum(soft_3in4_pen))
    m.Minimize(sum(obj_terms))

    # solve
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = float(TIME_LIMIT_SEC)
    solver.parameters.num_search_workers  = int(NUM_WORKERS)
    solver.parameters.linearization_level = 1
    solver.parameters.symmetry_level      = 2
    solver.parameters.log_to_stdout       = True
    solver.parameters.log_search_progress = True

    status = solver.Solve(m)
    print("Status:", solver.StatusName(status))
    report = f"Status: {solver.StatusName(status)}\nWall time: {solver.WallTime():.2f}s\nResponse:\n{solver.ResponseStats()}\n"
    (BASE/"solve_report_fast.txt").write_text(report, encoding="utf-8")
    print(report)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        # extract schedule
        assign = {}
        for g in games:
            gid = g["id"]
            for d in D:
                if solver.Value(x[(gid,d)]) == 1:
                    assign[gid] = d; break
        # save CSV
        out_csv = BASE / "Scheduled_FAST.csv"
        with out_csv.open("w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["day","away","home","venue","game_id"])
            for g in sorted(games, key=lambda G: (assign.get(G["id"], 10**9), G["id"])):
                w.writerow([assign[g["id"]], g["away"], g["home"], g["venue"], g["id"]])
        print(f"Saved → {out_csv}")
    else:
        print("No solution produced; increase DAYS_HORIZON or set HARD_3IN4=False temporarily.")

# run fast solver
solve_fast()


Sanity lower bounds → team-days: 82, 3-in-4: 110, venue: 82.  Horizon=180.
Status: FEASIBLE
Status: FEASIBLE
Wall time: 309.15s
Response:
CpSolverResponse summary:
status: FEASIBLE
objective: 265747
best_bound: 60470
integers: 241751
booleans: 226678
conflicts: 171
branches: 1667213
propagations: 19027672
integer_propagations: 21687644
restarts: 233459
lp_iterations: 0
walltime: 309.155
usertime: 309.155
deterministic_time: 109.204
gap_integral: 1209.2
solution_fingerprint: 0x8d8ece4afefa43c3


Saved → C:\Users\Tanmay\Downloads\UNIVERSITY OF MELBOURNE STUDY FOLDER\Scheduling and Optimisation\Scheduled_FAST.csv
